In [1]:
import os,sys,tqdm
import numpy as np
import tensorflow as tf
import tensorflow.keras
from tensorflow.keras.datasets import *
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras.preprocessing.text import *

from collections import Counter
import pandas as pd
import shutil
import pickle
import gc
# os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
# The GPU id to use, usually either "0" or "1"
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [2]:
def basic_statistics(all_length):
    '''
    input: length list of elements e.g.[1,1,1,3,5,9,4,2,1,3,54,78,5...]
    output1: mean、std、mode、min、q1、median(q2)、q3、max、iqr、outlier、far out
    output2: statistics graph、10%~90% form
    '''
    stat_dict = {}
    stat_dict['mean'] = np.mean(all_length)
    stat_dict['std'] = np.std(all_length)
    stat_dict['mode'] = np.argmax(np.bincount(all_length))
    stat_dict['min'] = np.min(all_length)
    stat_dict['q1'] = np.quantile(all_length,0.25)
    stat_dict['median'] = np.quantile(all_length,0.5)
    stat_dict['q3'] = np.quantile(all_length,0.75)
    stat_dict['max'] = np.max(all_length)
    stat_dict['iqr'] = stat_dict['q3'] - stat_dict['q1']
    stat_dict['outlier'] = stat_dict['q3'] + 1.5*stat_dict['iqr']
    stat_dict['far_out'] = stat_dict['q3'] + 3*stat_dict['iqr']
    for i in [10,20,30,40,50,60,70,80,90,100]:
        stat_dict[str(i)+'%'] = np.percentile(all_length,i)
    return pd.DataFrame.from_dict(stat_dict,orient='index',columns=['length'])

In [3]:
max_words = 8352#8352 #Top most frequent words to consider. Any less frequent word will appear as oov_char value in the sequence data.
max_length = 360#360

In [4]:
word_index = reuters.get_word_index()
print('all_words#:',len(word_index))
(x_train, y_train), (x_test, y_test) = reuters.load_data(num_words=max_words,maxlen=max_length,
                                                         test_split=0.2,seed=830913)
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')

all_words#: 30979
8260 train sequences
2066 test sequences


In [13]:
# train_len = [len(x) for x in x_train]
# test_len = [len(x) for x in x_test]
# all_len = train_len
# all_len.extend(test_len)
# basic_statistics(all_len)

,length
mean,145.964197
std,145.878476
mode,17.000000
min,2.000000
q1,60.000000
median,95.000000
q3,180.000000
max,2376.000000
iqr,120.000000
outlier,360.000000


In [15]:
# df = pd.DataFrame(all_len)
# df.to_excel('./results/length_dist.xlsx', header=False, index=False)
# df

In [21]:
# train_words = []
# for x in x_train:
#     train_words.extend(x)
# test_words = []
# for x in x_test:
#     test_words.extend(x)
# all_words = train_words
# all_words.extend(test_words)
# all_statistcs = Counter(all_words)
# all_statistcs

Counter({1: 11228,
         53: 4213,
         352: 647,
         26: 8451,
         14: 15015,
         279: 801,
         39: 5818,
         72: 3091,
         4497: 26,
         18: 11039,
         83: 2597,
         5291: 21,
         88: 2381,
         5397: 20,
         11: 20141,
         3412: 37,
         19: 10755,
         151: 1363,
         230: 962,
         831: 253,
         15: 13329,
         165: 1232,
         318: 707,
         3780: 33,
         124: 1676,
         1527: 117,
         1424: 128,
         35: 6588,
         5302: 20,
         12: 16668,
         17: 11191,
         486: 459,
         341: 663,
         142: 1466,
         255: 870,
         219: 997,
         429: 528,
         68: 3363,
         146: 1402,
         252: 882,
         191: 1098,
         15448: 3,
         3631: 35,
         2283: 65,
         71: 3120,
         10: 29581,
         342: 660,
         49: 4565,
         1977: 80,
         324: 695,
         27: 8311,
         9222: 

In [28]:
# df = pd.DataFrame.from_dict(dict(all_statistcs), orient = 'index')
# df.to_excel('./results/words_dist2.xlsx', header=False, index=True)
# df

,0
1,11228
53,4213
352,647
26,8451
14,15015
...,...
24452,1
18567,2
27222,1
26864,1


In [5]:
trainX = tf.keras.preprocessing.sequence.pad_sequences(x_train,maxlen=max_length,padding='post',value=0)
testX = tf.keras.preprocessing.sequence.pad_sequences(x_test,maxlen=max_length,padding='post',value=0)
print(trainX.shape, testX.shape)

(8260, 360) (2066, 360)


In [563]:
hidden_dim = 128
# do = 0.1

## Graph execution
### Embedder

In [42]:
int_id = Input(shape=(max_length,), dtype='int32', name='int_ids') # 輸入的api funvtion name ID
int_ids = Masking(mask_value=0)(int_id)
sent_emb = Embedding(max_words, hidden_dim,input_length=max_length
                    ,trainable=True,name='glove_emb')(int_ids) 

### Encoder

In [43]:
rnn = GRU(int(hidden_dim/2),return_sequences=True,return_state=False,name='common_extract'
                      ,trainable=True)(sent_emb)
rnn = BatchNormalization(name='bn')(rnn)


### Filter

In [44]:
fil = TimeDistributed(Dense(1,activation='sigmoid',
                             name='filter_out'),name='TD2')(rnn)

### Classfier

In [47]:
mul = Multiply()([fil,sent_emb])
clf = LSTM(int(hidden_dim/2),dropout=do,recurrent_dropout=do,name='lstm')(mul)
clf = BatchNormalization(name='bn3')(clf)
clf = Dense(max(y_train)+1,activation='softmax',name='clf')(clf)

## Compile

In [48]:
model = Model(inputs=int_id, outputs = clf)
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
int_ids (InputLayer)            [(None, 358)]        0                                            
__________________________________________________________________________________________________
masking_1 (Masking)             (None, 358)          0           int_ids[0][0]                    
__________________________________________________________________________________________________
glove_emb (Embedding)           (None, 358, 128)     1280000     masking_1[0][0]                  
__________________________________________________________________________________________________
common_extract (GRU)            (None, 358, 64)      37248       glove_emb[0][0]                  
______________________________________________________________________________________________

In [53]:
# loss
import keras.backend as K
def custom_objective(layer):
    return K.sum(layer.output)
#     return K.sum(layer.output)
# kk = tf.keras.backend.ea
model.compile(loss=custom_objective(model.get_layer(name='TD2')),optimizer='adam')

Using TensorFlow backend.


OperatorNotAllowedInGraphError: using a `tf.Tensor` as a Python `bool` is not allowed in Graph execution. Use Eager execution or decorate this function with @tf.function.

## Eager Execution

In [52]:
# whole model
do = 0
init = tensorflow.keras.initializers.Ones()
class base_model(Model):
    def __init__(self):
        super(base_model, self).__init__()
        self.mask = Masking(mask_value=0)
        self.emb = Embedding(max_words, hidden_dim,input_length=max_length
                    ,trainable=True,name='glove_emb')
        self.rnn1 = GRU(int(hidden_dim/2),return_sequences=True,return_state=False,name='common_extract'
                      ,trainable=True)
        self.bn1 = BatchNormalization(name='bn1')
        self.fil = Dense(1,activation='hard_sigmoid',kernel_initializer=init,bias_initializer=init,name='filter_out')
        #self.fil = TimeDistributed(Dense(1,activation='sigmoid', name='filter_out'),name='TD2')
        self.mul = Multiply()
        self.rnn2 = Bidirectional(GRU(int(hidden_dim/2),dropout=do,recurrent_dropout=do,name='lstm'))
        self.rnn3 = LSTM(int(hidden_dim/2))
        self.bn2 = BatchNormalization(name='bn2')
        self.out = Dense(max(y_train)+1,activation='softmax',name='clf')
    def transform(self,x):
        return tf.math.round(x)
    def call(self,x):
        x = self.mask(x)
        x1 = self.emb(x)
        x = self.rnn1(x1)
        x = self.bn1(x)
        y = self.fil(x)
        y1 = self.transform(y)
        x2 = self.mul([y1,x1])
        x = self.rnn2(x2) #x
        x = self.bn2(x)
        y2 = self.out(x)
        return y,y1,y2
        #return y,y1,y2,x2
        
model = base_model()

In [995]:
# partial1 model
init_w = tensorflow.keras.initializers.Constant(value=1.0) #portyion=0.6, w=0.9, b = 0.8-0.85 (0.83從0開始)
init_b = tensorflow.keras.initializers.Constant(value=1.0) #w=1 ; b=0.499, portion=1
def onezero(x):
    portion = 0.6#0.6#0.6 #0.6~1
    z = tf.where(x>=1.0, x - x + 1.0, x)
    y = tf.where(z<=0.0, z - z + 0.0, portion*z)
    return y

class base_model_1(Model):
    def __init__(self):
        super(base_model_1, self).__init__()
        self.mask = Masking(mask_value=0)
        self.emb = Embedding(max_words, hidden_dim,input_length=max_length
                    ,trainable=True,name='glove_emb')
        self.rnn1 = GRU(int(hidden_dim/4),return_sequences=True,return_state=False,name='common_extract'
                      ,trainable=True)
        self.att = Attention(name='selfatt')
        self.bn1 = BatchNormalization(name='bn1')
        #self.fil = Dense(1,activation=onezero,name='filter_out')
        self.fil = TimeDistributed(Dense(1,activation=onezero,kernel_initializer=init_w,bias_initializer=init_b, name='filter_out'),name='TD2') #relu/linear/step function

    def call(self,x):
        x = self.mask(x)
        x1 = self.emb(x)
        x = self.att([x1,x1])
        #x = self.rnn1(x1)
        #x = self.bn1(x)
        y = self.fil(x)
        return x1,y

model1 = base_model_1()
#phase2
# model3 = load_model(saveP)

# model3 = base_model_1()
# model3.load_weights('./model/2019110501/model1')#,by_name=True)
# model1.emb.set_weights(model3.emb.get_weights())

In [996]:
# partial2 model
class base_model_2(Model):
    def __init__(self):
        super(base_model_2, self).__init__()
        self.mul = Multiply()
        self.rnn2 = Bidirectional(GRU(int(hidden_dim/2),dropout=do,recurrent_dropout=do,name='lstm'))
        self.rnn3 = GRU(int(hidden_dim/2))
        self.bn2 = BatchNormalization(name='bn2')
        self.out = Dense(max(y_train)+1,activation='softmax',name='clf')

    def call(self,x1,y1):
        x2 = self.mul([y1,x1])
        x = self.rnn3(x2) #x2 #y1=weight|binary
        x = self.bn2(x)
        y2 = self.out(x)
        return y2
    
model2 = base_model_2()

In [448]:
'''
# TEST
x = tf.random.uniform((32, 6))
out1,out2 = model1(x)
out2 = tf.squeeze(out2,axis=-1)
# out2 = out2.astype('float32')
# out2 = tf.dtypes.cast(out2, tf.float8)
# print(out2.shape)
yy = tf.matmul(out2,kk)
print(yy[0].shape)
tf.where(yy[0]==0,1,0)

gg = tf.range(-5,5)
gg = tf.expand_dims(gg,axis=0)
gg = tf.keras.backend.repeat_elements(gg,rep=32,axis=0)
tf.where(gg==1,gg,0)

kk = tf.Variable(np.array([[1.0,1.0,1.0,0.0,0.0,0.0],[0.0,1.0,1.0,1.0,0.0,0.0],[0.0,0.0,1.0,1.0,1.0,0.0],
                           [0.0,0.0,0.0,1.0,1.0,1.0]]).T,dtype='float32')
kk = tf.expand_dims(kk,axis=0)
kk = tf.keras.backend.repeat_elements(kk,rep=1,axis=0)
kk.shape
'''

(32, 4)


<tf.Tensor: id=5770753, shape=(32, 4), dtype=int32, numpy=
array([[1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1]], dtype=int32)>

In [896]:
記得要跑到一個好的data
# batch_size = 128 #,reshuffle_each_iteration=True
# train_ds = tf.data.Dataset.from_tensor_slices((trainX,y_train)).shuffle(trainX.shape[0]).batch(batch_size)
# valid_ds = tf.data.Dataset.from_tensor_slices((testX,y_test)).batch(batch_size)

In [997]:
seq_num = 2 #連續幾個才叫做連續，要改modify_idx看有幾個

arr_len = max_length - seq_num + 1
seq_arr = []
for i in range(arr_len):
    ori_np = np.array([0]*max_length)
    modify_idx = [i,i+1] #要跟著seq_num改
    ori_np[modify_idx]=1
    seq_arr.append(ori_np)
seq_arr = np.array(seq_arr)
seq_mask = tf.Variable(seq_arr.T,dtype='float32')
seq_mask = tf.expand_dims(seq_mask,axis=0)
seq_mask

<tf.Tensor: id=13834004, shape=(1, 360, 359), dtype=float32, numpy=
array([[[1., 0., 0., ..., 0., 0., 0.],
        [1., 1., 0., ..., 0., 0., 0.],
        [0., 1., 1., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 1., 1., 0.],
        [0., 0., 0., ..., 0., 1., 1.],
        [0., 0., 0., ..., 0., 0., 1.]]], dtype=float32)>

In [998]:
def loss_object1(predictions): #filter loss
    mask = tf.math.logical_not(tf.math.equal(predictions, 0))
    loss_ = tf.reduce_mean(predictions)
    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask
    return tf.reduce_mean(loss_)
def one_percentage(predictions): #1 num
    mask = tf.math.logical_not(tf.math.equal(predictions, 0))
    loss_ = tf.reduce_mean(predictions)
    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask
    return tf.reduce_mean(loss_)
def seq_loss(predictions):
    mask = tf.math.logical_not(tf.math.equal(predictions, 0))
    predictions = tf.squeeze(predictions,axis=-1)
    results = tf.matmul(predictions,seq_mask)
    results = tf.where(results==seq_num,1.0,0.0)
    loss_ = tf.reduce_mean(results)
    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask
    return tf.reduce_mean(loss_)
loss_object2 = tf.keras.losses.SparseCategoricalCrossentropy() #clf loss

optimizer1 = tf.keras.optimizers.Nadam(learning_rate=0.001)
optimizer2 = tf.keras.optimizers.RMSprop()

train_loss = tf.keras.metrics.Mean(name='train_loss') #total_loss
train_accloss = tf.keras.metrics.Mean(name='train_accloss')#loss_acc
train_filloss = tf.keras.metrics.Mean(name='train_filloss') #loss_filter
train_seqloss = tf.keras.metrics.Mean(name='train_seqloss') #loss_seq
train_ones = tf.keras.metrics.Mean(name='train_ones') #ones_num
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy') #acc_rate

test_loss = tf.keras.metrics.Mean(name='test_loss') #total_loss
test_accloss = tf.keras.metrics.Mean(name='test_accloss')#loss_acc
test_filloss = tf.keras.metrics.Mean(name='test_filloss') #loss_filter
test_seqloss = tf.keras.metrics.Mean(name='test_seqloss')
test_ones = tf.keras.metrics.Mean(name='test_ones') #ones_num
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

In [999]:
# seperate partial model
alpha = 0.00001 #pahse1: -0.1 / 0.0 ; phase2: 0.01~0.05~0.1 有1-就是希望1越多，沒1-就是希望0越多1越少
beta = 1.0 #clf loss 越大越要求分好
gamma = 0.0 #seqloss 越大越要求連續
#-0.001 / 1.0 / 1.0

@tf.function
def train_step(x,yc):
    with tf.GradientTape(persistent=False) as tape:
        emb, pred_imp = model1(x)
        #loss1 = alpha*loss_object1(pred_imp) #phase1
        #pred_imp2 = tf.math.round(pred_imp)
        #pred_imp3 = tf.clip_by_value(pred_imp,clip_value_max=1,clip_value_min=0)
        pred_imp2 = tf.math.round(pred_imp)
        loss1 = loss_object1(pred_imp) #有1-就是希望1越多，沒1-就是希望0越多1越少 #pahse2:alpha*loss_object1(pred_imp) ; phase1: alpha*(1-loss_object1(pred_imp))
        pred_cat = model2(emb,pred_imp2) #pahse1: pred_imp; phase2; pred_imp2
        loss2 = loss_object2(yc, pred_cat)
        loss3 = 1-seq_loss(pred_imp2)
        loss = alpha*loss1 + beta*loss2 + gamma*loss3
    trainable_variable = model1.trainable_variables
    trainable_variable.extend(model2.trainable_variables)
    gradients = tape.gradient(loss,trainable_variable)
    optimizer1.apply_gradients(zip(gradients,trainable_variable))
    
    train_loss(loss) #total_loss
    train_filloss(loss1)
    train_accloss(loss2)
    train_seqloss(loss3) #loss_seq
    train_accuracy(yc, pred_cat) #acc_rate
    ones = one_percentage(pred_imp2) #pred_imp2
    train_ones(ones) #ones_num
    
    
@tf.function
def test_step(x,yc):
    emb, pred_imp = model1(x)
    #loss1 = alpha*loss_object1(pred_imp) #phase1
    #pred_imp2 = tf.math.round(pred_imp)
    #pred_imp3 = tf.clip_by_value(pred_imp,clip_value_max=1,clip_value_min=0)
    pred_imp2 = tf.math.round(pred_imp)
    loss1 = loss_object1(pred_imp) #phase2
    pred_cat = model2(emb,pred_imp2) #phase1: pred_imp ; phase2:pred_imp2
    loss2 = loss_object2(yc, pred_cat)
    loss3 = 1-seq_loss(pred_imp2)
    #t_loss = loss1 + loss2
    t_loss = alpha*loss1 + beta*loss2 + gamma*loss3
    
    test_loss(t_loss)
    test_filloss(loss1)
    test_accloss(loss2)
    test_seqloss(loss3)
    test_accuracy(yc, pred_cat)
    t_ones = one_percentage(pred_imp2) #pred_imp2
    test_ones(t_ones)
    

In [331]:
'''
#AIO
alpha = 0.1
beta = 1
gamma = 0
@tf.function
def train_step(x,yc):
    with tf.GradientTape(persistent=True) as tape: #persistent=True
        pred_imp,pred_round , pred_cat = model(x)
#         pred_cat = model(x)
#         loss = alpha*loss_object1(pred_imp) + loss_object2(yc,pred_cat)
        loss1 = alpha*loss_object1(pred_imp)
        loss2 = beta*loss_object2(yc,pred_cat)
        loss = loss_object2(yc,pred_cat)
#     gradients = tape.gradient(loss, model.trainable_variables)
    grad1 = tape.gradient(loss1, model.trainable_variables)
    grad2 = tape.gradient(loss2, model.trainable_variables)
#     optimizer1.apply_gradients(zip(gradients, model.trainable_variables))
    optimizer1.apply_gradients(zip(grad1, model.trainable_variables))
    optimizer2.apply_gradients(zip(grad2, model.trainable_variables))
#     with tf.GradientTape() as tape:
#         pred_imp , pred_cat = model(x)
#         loss2 = loss_object2(yc,pred_cat)
#         loss = alpha*loss_object1(pred_imp) + loss_object2(yc,pred_cat)
#     grad2 = tape.gradient(loss2, model.trainable_variables)
#     optimizer2.apply_gradients(zip(grad2, model.trainable_variables))

    train_loss(loss)
    train_accuracy(yc, pred_cat)
    ones = one_percentage(pred_round)
    train_ones(ones)
    
@tf.function
def test_step(x,yc):
    pred_imp,pred_round, pred_cat = model(x)
#     pred_cat = model(x)
    t_loss = alpha*loss_object1(pred_imp) + loss_object2(yc,pred_cat) 
#     t_loss = loss_object2(yc,pred_cat)
    
    test_loss(t_loss)
    test_accuracy(yc, pred_cat)
    t_ones = one_percentage(pred_round)
    test_ones(t_ones)
'''    

In [1000]:
# EPOCHS = 2000
DateID = '2019111001'

saveP1 = './model/'+DateID+'/model1'
saveP2 = './model/'+DateID+'/model2'
train_loss_acc = []
train_loss_filter = []
train_loss_seq = []
train_weighted_loss = []
train_acc_rate = []
train_ones_num = []

test_loss_acc = []
test_loss_filter = []
test_loss_seq = []
test_weighted_loss = []
test_acc_rate = []
test_ones_num = []

# signature_dict = {'att':model1.att}

gc.collect()
best_clf = 0.0
for epoch in range(EPOCHS):
    for text, labels in train_ds:
        train_step(text, labels)

    for test_text, test_labels in valid_ds:
        test_step(test_text, test_labels)

    template = 'Epoch {}, Total Loss: {}, Clf Loss: {}, Filter Loss: {}, Seq Loss: {}, Accuracy Rate: {:5.2f}%, Ones Portion: {}, \
            Test_Total_Loss: {}, Test_Clf_Loss: {}, Test_Filter_Loss: {}, TEST_Seq_Loss: {}, Test_Accuracy_Rate: {:5.2f}%, Test_Ones_Portion: {}'
    print(template.format(epoch+1,train_loss.result(),
                          train_accloss.result(),train_filloss.result(),train_seqloss.result(),
                        train_accuracy.result()*100,train_ones.result(),
                        test_loss.result(),
                        test_accloss.result(),test_filloss.result(),test_seqloss.result(),
                        test_accuracy.result()*100,test_ones.result(),
                        ))

    train_loss_acc.append( train_accloss.result().numpy())
    train_loss_filter.append( train_filloss.result().numpy())
    train_loss_seq.append( train_seqloss.result().numpy())
    train_weighted_loss.append( train_loss.result().numpy())
    train_acc_rate.append( train_accuracy.result().numpy())
    train_ones_num.append( train_ones.result().numpy())
    
    test_loss_acc.append( test_accloss.result().numpy())
    test_loss_filter.append( test_filloss.result().numpy())
    test_loss_seq.append( test_seqloss.result().numpy())
    test_weighted_loss.append( test_loss.result().numpy())
    test_acc_rate.append( test_accuracy.result().numpy())
    test_ones_num.append( test_ones.result().numpy())
    if best_clf<=test_accuracy.result()*100:
        #tf.saved_model.save(model1,saveP1+'_all')
        #model1.save(saveP1,save_format='h5')
        #model2.save(saveP2,save_format='h5')
        #tf.saved_model.save(model2,saveP2+'_all')
        model1.save_weights(saveP1,save_format='tf')
        model2.save_weights(saveP2,save_format='tf')
        best_clf = test_accuracy.result()*100
        print('===MODEL WEIGHTS SAVED===',saveP1,saveP2)
    # Reset the metrics for the next epoch
    train_loss.reset_states()
    train_accloss.reset_states()
    train_filloss.reset_states()
    train_seqloss.reset_states()
    train_ones.reset_states()
    train_accuracy.reset_states()
    
    test_loss.reset_states()
    test_accloss.reset_states()
    test_filloss.reset_states()
    test_seqloss.reset_states()
    test_ones.reset_states()
    test_accuracy.reset_states()

Epoch 1, Total Loss: 2.6837620735168457, Clf Loss: 2.683756113052368, Filter Loss: 0.5167008638381958, Seq Loss: 0.4507025182247162, Accuracy Rate: 36.72%, Ones Portion: 0.5494323968887329,             Test_Total_Loss: 2.372053861618042, Test_Clf_Loss: 2.372049331665039, Test_Filter_Loss: 0.45908623933792114, TEST_Seq_Loss: 0.9874978065490723, Test_Accuracy_Rate: 37.17%, Test_Ones_Portion: 0.012629319913685322
===MODEL WEIGHTS SAVED=== ./model/2019111001/model1 ./model/2019111001/model2
Epoch 2, Total Loss: 2.3417890071868896, Clf Loss: 2.341784954071045, Filter Loss: 0.40750208497047424, Seq Loss: 0.9957640171051025, Accuracy Rate: 37.72%, Ones Portion: 0.004312779288738966,             Test_Total_Loss: 2.358151912689209, Test_Clf_Loss: 2.3581480979919434, Test_Filter_Loss: 0.36273330450057983, TEST_Seq_Loss: 0.9928811192512512, Test_Accuracy_Rate: 37.17%, Test_Ones_Portion: 0.007338809780776501
===MODEL WEIGHTS SAVED=== ./model/2019111001/model1 ./model/2019111001/model2
Epoch 3, Tot

Epoch 18, Total Loss: 1.281201958656311, Clf Loss: 1.281201720237732, Filter Loss: 0.021081801503896713, Seq Loss: 0.9984032511711121, Accuracy Rate: 68.56%, Ones Portion: 0.0036298055201768875,             Test_Total_Loss: 1.3450368642807007, Test_Clf_Loss: 1.3450367450714111, Test_Filter_Loss: 0.022237766534090042, TEST_Seq_Loss: 0.9985226988792419, Test_Accuracy_Rate: 68.01%, Test_Ones_Portion: 0.0037246590945869684
===MODEL WEIGHTS SAVED=== ./model/2019111001/model1 ./model/2019111001/model2
Epoch 19, Total Loss: 1.245874285697937, Clf Loss: 1.2458739280700684, Filter Loss: 0.019385386258363724, Seq Loss: 0.9986774325370789, Accuracy Rate: 69.93%, Ones Portion: 0.003209840739145875,             Test_Total_Loss: 1.3201289176940918, Test_Clf_Loss: 1.3201286792755127, Test_Filter_Loss: 0.02109520696103573, TEST_Seq_Loss: 0.9986681938171387, Test_Accuracy_Rate: 69.07%, Test_Ones_Portion: 0.0035061119124293327
===MODEL WEIGHTS SAVED=== ./model/2019111001/model1 ./model/2019111001/model2

Epoch 36, Total Loss: 0.9136047959327698, Clf Loss: 0.9136046767234802, Filter Loss: 0.007086764555424452, Seq Loss: 0.9997739195823669, Accuracy Rate: 78.05%, Ones Portion: 0.0011944860452786088,             Test_Total_Loss: 1.3089519739151, Test_Clf_Loss: 1.3089519739151, Test_Filter_Loss: 0.008036519400775433, TEST_Seq_Loss: 0.9997961521148682, Test_Accuracy_Rate: 70.52%, Test_Ones_Portion: 0.0011601769365370274
Epoch 37, Total Loss: 1.0018565654754639, Clf Loss: 1.0018564462661743, Filter Loss: 0.0065278965048491955, Seq Loss: 0.9997951984405518, Accuracy Rate: 76.15%, Ones Portion: 0.0011082375422120094,             Test_Total_Loss: 1.2903501987457275, Test_Clf_Loss: 1.290350079536438, Test_Filter_Loss: 0.007550009060651064, TEST_Seq_Loss: 0.9998058080673218, Test_Accuracy_Rate: 71.01%, Test_Ones_Portion: 0.0011267594527453184
Epoch 38, Total Loss: 0.9743315577507019, Clf Loss: 0.9743313789367676, Filter Loss: 0.006111369002610445, Seq Loss: 0.9998085498809814, Accuracy Rate: 76.3

Epoch 56, Total Loss: 0.7725915908813477, Clf Loss: 0.7725915312767029, Filter Loss: 0.0027806132566183805, Seq Loss: 0.9998646378517151, Accuracy Rate: 80.59%, Ones Portion: 0.0009178075706586242,             Test_Total_Loss: 1.3855739831924438, Test_Clf_Loss: 1.3855739831924438, Test_Filter_Loss: 0.0030603427439928055, TEST_Seq_Loss: 0.9998757839202881, Test_Accuracy_Rate: 70.23%, Test_Ones_Portion: 0.0009120352915488183
Epoch 57, Total Loss: 0.7711397409439087, Clf Loss: 0.7711397409439087, Filter Loss: 0.0026582572609186172, Seq Loss: 0.9998688697814941, Accuracy Rate: 80.86%, Ones Portion: 0.0008980383863672614,             Test_Total_Loss: 1.3605071306228638, Test_Clf_Loss: 1.3605071306228638, Test_Filter_Loss: 0.0029491723980754614, TEST_Seq_Loss: 0.9998776912689209, Test_Accuracy_Rate: 70.86%, Test_Ones_Portion: 0.0008999468991532922
Epoch 58, Total Loss: 0.7713317275047302, Clf Loss: 0.7713317275047302, Filter Loss: 0.0025806783232837915, Seq Loss: 0.9998698234558105, Accuracy

Epoch 76, Total Loss: 0.6572074294090271, Clf Loss: 0.6572074294090271, Filter Loss: 0.00183676031883806, Seq Loss: 0.999883234500885, Accuracy Rate: 83.64%, Ones Portion: 0.0008576525142416358,             Test_Total_Loss: 1.5156594514846802, Test_Clf_Loss: 1.5156594514846802, Test_Filter_Loss: 0.0019412445835769176, TEST_Seq_Loss: 0.9998890161514282, Test_Accuracy_Rate: 70.62%, Test_Ones_Portion: 0.0008582811569795012
Epoch 77, Total Loss: 0.6439540386199951, Clf Loss: 0.6439540386199951, Filter Loss: 0.0017792695434764028, Seq Loss: 0.9998856782913208, Accuracy Rate: 83.91%, Ones Portion: 0.0008422631653957069,             Test_Total_Loss: 1.5313372611999512, Test_Clf_Loss: 1.5313372611999512, Test_Filter_Loss: 0.0018990052631124854, TEST_Seq_Loss: 0.9998907446861267, Test_Accuracy_Rate: 69.85%, Test_Ones_Portion: 0.0008529888582415879
Epoch 78, Total Loss: 0.6376115083694458, Clf Loss: 0.6376115083694458, Filter Loss: 0.001779199461452663, Seq Loss: 0.9998842477798462, Accuracy Rat

...

Epoch 1912, Total Loss: 0.9069933891296387, Clf Loss: 0.9069933891296387, Filter Loss: 6.166673847474158e-05, Seq Loss: 0.9999992847442627, Accuracy Rate: 76.49%, Ones Portion: 4.526103293756023e-05,             Test_Total_Loss: 3.490905523300171, Test_Clf_Loss: 3.490905523300171, Test_Filter_Loss: 5.8303859987063333e-05, TEST_Seq_Loss: 0.9999987483024597, Test_Accuracy_Rate: 54.94%, Test_Ones_Portion: 4.357654324849136e-05
Epoch 1913, Total Loss: 0.8984454870223999, Clf Loss: 0.8984454870223999, Filter Loss: 6.171459244797006e-05, Seq Loss: 0.999998927116394, Accuracy Rate: 76.76%, Ones Portion: 4.544130206340924e-05,             Test_Total_Loss: 3.492055654525757, Test_Clf_Loss: 3.492055654525757, Test_Filter_Loss: 5.82713691983372e-05, TEST_Seq_Loss: 0.9999987483024597, Test_Accuracy_Rate: 54.60%, Test_Ones_Portion: 4.3667543650371954e-05
Epoch 1914, Total Loss: 0.9000000953674316, Clf Loss: 0.9000000953674316, Filter Loss: 6.170606502564624e-05, Seq Loss: 0.9999991655349731, Accura

Epoch 1932, Total Loss: 0.8915810585021973, Clf Loss: 0.8915810585021973, Filter Loss: 6.095484786783345e-05, Seq Loss: 0.9999992847442627, Accuracy Rate: 76.55%, Ones Portion: 4.4968230213271454e-05,             Test_Total_Loss: 3.5116868019104004, Test_Clf_Loss: 3.5116868019104004, Test_Filter_Loss: 5.748913827119395e-05, TEST_Seq_Loss: 0.9999987483024597, Test_Accuracy_Rate: 54.84%, Test_Ones_Portion: 4.3188509152969345e-05
Epoch 1933, Total Loss: 0.8906384110450745, Clf Loss: 0.8906384110450745, Filter Loss: 6.067549475119449e-05, Seq Loss: 0.9999992847442627, Accuracy Rate: 76.80%, Ones Portion: 4.4869659177493304e-05,             Test_Total_Loss: 3.549673318862915, Test_Clf_Loss: 3.549673318862915, Test_Filter_Loss: 5.7392262533539906e-05, TEST_Seq_Loss: 0.9999987483024597, Test_Accuracy_Rate: 54.55%, Test_Ones_Portion: 4.3188509152969345e-05
Epoch 1934, Total Loss: 0.9057400226593018, Clf Loss: 0.9057400226593018, Filter Loss: 6.053584729670547e-05, Seq Loss: 0.9999991655349731,

Epoch 1952, Total Loss: 1.0186808109283447, Clf Loss: 1.0186808109283447, Filter Loss: 5.868067455594428e-05, Seq Loss: 0.9999992847442627, Accuracy Rate: 73.70%, Ones Portion: 4.41046868218109e-05,             Test_Total_Loss: 3.490201950073242, Test_Clf_Loss: 3.490201950073242, Test_Filter_Loss: 5.553543451242149e-05, TEST_Seq_Loss: 0.9999987483024597, Test_Accuracy_Rate: 54.84%, Test_Ones_Portion: 4.199586692266166e-05
Epoch 1953, Total Loss: 0.9506257772445679, Clf Loss: 0.9506257772445679, Filter Loss: 5.8833851653616875e-05, Seq Loss: 0.9999992847442627, Accuracy Rate: 74.77%, Ones Portion: 4.421058838488534e-05,             Test_Total_Loss: 3.447364330291748, Test_Clf_Loss: 3.447364330291748, Test_Filter_Loss: 5.546199099626392e-05, TEST_Seq_Loss: 0.9999987483024597, Test_Accuracy_Rate: 54.65%, Test_Ones_Portion: 4.203299249638803e-05
Epoch 1954, Total Loss: 0.925582766532898, Clf Loss: 0.925582766532898, Filter Loss: 5.8659799833549187e-05, Seq Loss: 0.9999994039535522, Accurac

Epoch 1972, Total Loss: 0.9262222647666931, Clf Loss: 0.9262222647666931, Filter Loss: 5.4774704040028155e-05, Seq Loss: 0.9999992847442627, Accuracy Rate: 75.90%, Ones Portion: 4.4124644773546606e-05,             Test_Total_Loss: 3.520308017730713, Test_Clf_Loss: 3.520308017730713, Test_Filter_Loss: 5.248851448413916e-05, TEST_Seq_Loss: 0.9999988675117493, Test_Accuracy_Rate: 54.31%, Test_Ones_Portion: 4.195808287477121e-05
Epoch 1973, Total Loss: 0.9085096120834351, Clf Loss: 0.9085096120834351, Filter Loss: 5.4750347771914676e-05, Seq Loss: 0.9999992847442627, Accuracy Rate: 76.09%, Ones Portion: 4.398608871269971e-05,             Test_Total_Loss: 3.504800796508789, Test_Clf_Loss: 3.504800796508789, Test_Filter_Loss: 5.241862891125493e-05, TEST_Seq_Loss: 0.9999988675117493, Test_Accuracy_Rate: 55.37%, Test_Ones_Portion: 4.181812619208358e-05
Epoch 1974, Total Loss: 0.8957837820053101, Clf Loss: 0.8957837820053101, Filter Loss: 5.4753738368162885e-05, Seq Loss: 0.9999992847442627, Ac

Epoch 1992, Total Loss: 0.9238305687904358, Clf Loss: 0.9238305687904358, Filter Loss: 5.394210165832192e-05, Seq Loss: 0.9999994039535522, Accuracy Rate: 75.81%, Ones Portion: 4.4150616304250434e-05,             Test_Total_Loss: 3.6313629150390625, Test_Clf_Loss: 3.6313629150390625, Test_Filter_Loss: 5.140623397892341e-05, TEST_Seq_Loss: 0.9999988675117493, Test_Accuracy_Rate: 54.07%, Test_Ones_Portion: 4.1719227738212794e-05
Epoch 1993, Total Loss: 0.9471123814582825, Clf Loss: 0.9471123814582825, Filter Loss: 5.3592753829434514e-05, Seq Loss: 0.9999991655349731, Accuracy Rate: 75.29%, Ones Portion: 4.3856627598870546e-05,             Test_Total_Loss: 3.6305859088897705, Test_Clf_Loss: 3.6305859088897705, Test_Filter_Loss: 5.111921564093791e-05, TEST_Seq_Loss: 0.9999988675117493, Test_Accuracy_Rate: 54.21%, Test_Ones_Portion: 4.160672688158229e-05
Epoch 1994, Total Loss: 0.9507086873054504, Clf Loss: 0.9507086873054504, Filter Loss: 5.3140603995416313e-05, Seq Loss: 0.999999165534973

### Store process

In [1001]:
save_dir = './results/'+DateID+'/'
if not os.path.isdir(save_dir):
    os.makedirs(save_dir, exist_ok=True)
saveR = save_dir + 'losses_metrics.xlsx'
data = {'train total loss':train_weighted_loss, 'train acc loss':train_loss_acc,
        'train filter loss':train_loss_filter,'train seq loss':train_loss_seq,
        'train acc rate':train_acc_rate, 'train ones num':train_ones_num,
        'test total loss':test_weighted_loss, 'test acc loss':test_loss_acc,
        'test filter loss':test_loss_filter, 'test seq loss': test_loss_seq,
        'test acc rate':test_acc_rate, 'test ones num':test_ones_num
       }
df = pd.DataFrame(data)
df.to_excel(saveR)
print(saveR)
df

./results/2019111001/losses_metrics.xlsx


,train total loss,train acc loss,train filter loss,train seq loss,train acc rate,train ones num,test total loss,test acc loss,test filter loss,test seq loss,test acc rate,test ones num
0,2.683762,2.683756,0.516701,0.450703,0.367191,0.549432,2.372054,2.372049,0.459086,0.987498,0.371733,0.012629
1,2.341789,2.341785,0.407502,0.995764,0.377240,0.004313,2.358152,2.358148,0.362733,0.992881,0.371733,0.007339
2,2.317389,2.317386,0.316606,0.989780,0.379540,0.011714,2.303567,2.303564,0.249808,0.989015,0.373669,0.013457
3,2.134195,2.134193,0.164130,0.994692,0.442615,0.007526,2.037394,2.037394,0.091437,0.996327,0.478703,0.005958
4,2.014271,2.014270,0.066487,0.997983,0.478087,0.003675,1.978067,1.978066,0.052519,0.997760,0.488867,0.004145
...,...,...,...,...,...,...,...,...,...,...,...,...
1995,1.261985,1.261985,0.000054,0.999999,0.688015,0.000044,3.517646,3.517646,0.000052,0.999999,0.542110,0.000042
1996,1.048910,1.048910,0.000054,0.999999,0.731961,0.000044,3.488565,3.488565,0.000052,0.999999,0.545015,0.000041
1997,1.037385,1.037385,0.000054,0.999999,0.741283,0.000043,3.495070,3.495070,0.000052,0.999999,0.545983,0.000041
1998,0.993772,0.993772,0.000054,0.999999,0.748910,0.000043,3.485764,3.485764,0.000052,0.999999,0.544531,0.000041


In [843]:
DateID

'2019110601'

In [714]:
np.array(train_weighted_loss[0])

array(None, dtype=object)

In [732]:
kk = np.array(test_accloss.result())
kk

array(2.3583207, dtype=float32)

In [735]:
test_accloss.result().numpy()

2.3583207

In [523]:
# model1.save('./model/emb_layer') #tf.keras.models.load_model
tf.saved_model.save(model1,'./model/emb_layer')
model3 = tf.keras.models.load_model('./model/emb_layer')

TypeError: unsupported callable

In [524]:
model3 = model1.load_weights('./model/emb_layer_weight')

AttributeError: 'CheckpointLoadStatus' object has no attribute 'summary'

In [527]:
model1.emb.set_weights(model3.emb.get_weights())

In [526]:
model3 = model1
model3.load_weights('./model/emb_layer_weight')
model3.summary()

Model: "base_model_1_58"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
masking_58 (Masking)         multiple                  0         
_________________________________________________________________
glove_emb (Embedding)        multiple                  2138112   
_________________________________________________________________


ValueError: You tried to call `count_params` on common_extract, but the layer isn't built. You can build it manually via: `common_extract.build(batch_input_shape)`.

* 在某些參數的情況下0 1是train不起來的但是weight可以
    * alpha = 0.0, beta=0.6
    * init_w = tensorflow.keras.initializers.Constant(value=0.9), init_b = tensorflow.keras.initializers.Constant(value=0.7)
* 0/1 with emb比較容易overfit。如果是weight的比較沒那麼嚴重

* 不用sigmoid或是hard_sigmoid。改良relu + linear，並拆成兩個model，把round前面多加上clip
    * sigmoid中間的變化太快(一瞬間就會掉到0或是1)，改成relu在>0~無限大(linear為了還在0~1)再去clip再round，可以看到每個epoch的變化
* 1st phase的beta一定要>=0.6否則不會動，ones#都會是0
    * beta=1 (放0 1進去): weight init設成1也沒用，但把bias設成1就會一開始都是ones#=1了。weight=1 bias=0.2都匯市0，bias=0.3會是0.87。0.6/0.3都是0。0.8/0.3差不多是0.5(但是train很慢acc進步很慢就是了)。0.5/0.5是從0.01開始往上升 (0 1放進去會比較難train是因為它的變化量太大，一下就是有或沒有，所以clf可能學不好，但如果是weight每次gradient進步的都是一小點就會比較容易上升)
    * beta=0.6 (放0 1進去): 0.8/0.5都是0。0.9/0.8 從0.9一直到0。0.9/0.6差不多是從0.5但又有時候到0.7都是0.0(很難train，Nadam換個opt有時候沒用。EX變成adam 0.9/0.8才有0.96開始但如果0.9/0.75變成0開始。Rmsprop 0.9/0.8又是從0.95開始往下)。但如果都改成傳入weight就都沒問題。ones一開始大概0.5 weight平均，也不會卡住
* 建議: 先訓練embedding weight matrix，但是要看goal是要怎樣的matrix
* 其實他不管幾%都會train得很好，除非固定embedding，或是用更弱的clf
* 若設定alpha，就像是regularizer term (penalty)，設越大drop越多
* 一開始先很多ones，再越來越少個

* 同一個opt若加入transform就會train不起來
* 兩個不同的opt加入transform也會train不起來 (persistent、non-persis都不行)，且與BN無關